メモ
I91,95,84あたりは多い

<table>
  <thead>
  <tr>
    <td>更新日時</td>
    <td>変更内容</td>    
  </tr>
  </thead>
  <tr>
    <td>0611 1230</td>
    <td>locationあり、officerなし。良さげ。</td> 
  </tr>
  <tr>
    <td>0613 1000</td>
    <td>dayを追加。location/officerは追加</td> 
  </tr>
  <tr>
    <td>0613 1430</td>
    <td>dayを追加。location/officerは追加</td> 
  </tr>
  <tr>
    <td>0613 1730</td>
    <td>locationのうち小さいものを除去するなど。officerなし。良さげ。</td> 
  </tr>
  <tr>
    <td>0613 1800</td>
    <td>location(>50)、officer(>1000)を追加。安定して0.88でる。</td> 
  </tr>
</table>

In [138]:
import pandas as pd
import numpy as np
from datetime import datetime

In [154]:
df = pd.read_csv("test.csv")

以下、データの成形を行なっていく。

Violationに関しては、カンマ区切りにされているviolationの種類をstr.splitで分割した上で、全てのviolationの種類をまとめたリストを作成する。このリストをカラム名とし、各indexについて対応するviolationの値を1とする。

In [155]:
# violationの種類を検索
df_violation = df["violation"].str.split(',',expand=True)
violation_list = np.array([])

"""
for i in range(0,4):
    violation_list = np.append(violation_list,df_violation[0].unique())
violation_list = np.unique(violation_list)
"""

violation_list = np.array(['Cell phone', 'Equipment', 'License', 'Lights', 'Moving violation',
       'Other', 'Registration/plates', 'Safe movement', 'Seat belt',
       'Speeding', 'Stop sign/light'], dtype=object)

for list in violation_list:
    df["violation_"+list] = np.zeros(df.contraband_found.count())
    for i in range(0,4):
        df.loc[df_violation[i]==list,"violation_"+list]=1

In [156]:
# stop_durationを数値に直す
# 2回目以降の実行ではエラー
df.loc[df["stop_duration"]=="1-15 min","stop_duration"]=0
df.loc[df["stop_duration"]=="16-30 min","stop_duration"]=1
df.loc[df["stop_duration"]=="30+ min","stop_duration"]=2

# driver_genderを数値に直す
# 2回目以降の実行ではエラー
df.loc[df["driver_gender"]=="M","driver_gender"]=0
df.loc[df["driver_gender"]=="F","driver_gender"]=1

In [157]:
df['stop_time'] = pd.to_datetime(df['stop_time'])
df['stop_date'] = pd.to_datetime(df['stop_date'])
df["stop_weekday_name"] = df["stop_date"].dt.weekday_name
df["stop_month"] = df["stop_date"].dt.month    
df["stop_year"] = df["stop_date"].dt.year
df["stop_hour"] = df["stop_time"].dt.hour
df["stop_day"] = df["stop_date"].dt.day

In [158]:
df = pd.get_dummies(df,columns=["stop_month"], prefix='stop_month', prefix_sep='_')
df = pd.get_dummies(df,columns=["stop_year"], prefix='stop_year', prefix_sep='_')
df = pd.get_dummies(df,columns=["stop_weekday_name"], prefix='stop_day', prefix_sep='_')
df = pd.get_dummies(df,columns=["stop_hour"], prefix='stop_hour', prefix_sep='_')

df = pd.get_dummies(df,columns=["driver_race"],prefix='race',prefix_sep='_')

df = pd.get_dummies(df,columns=["search_type"], prefix='search', prefix_sep='_')

df = pd.get_dummies(df,columns=["county_name"], drop_first=True, prefix='county', prefix_sep='_')

In [159]:
df = df.drop(["stop_date","stop_time","search_type_raw","search_conducted","county_fips"
              ,"driver_race_raw","violation","violation_raw","state"
              ,"police_department","driver_age"],axis=1)

In [160]:
df["contraband_found"] = df["contraband_found"]*1

In [161]:
df["location_raw"] = df["location_raw"].fillna("N/A")
df["fine_grained_location"] = df["fine_grained_location"].fillna("N/A")

In [162]:
df["location_exit"] = 0
df.loc[df["fine_grained_location"].str.contains("X"),"location_exit"] = 1
df.loc[df["fine_grained_location"].str.contains("x"),"location_exit"] = 1
df.loc[df["fine_grained_location"].str.contains("sex"),"location_exit"] = 0

df["location_route"] = 0
df.loc[df["fine_grained_location"].str.contains("rt"),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("rte "),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("RT."),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("RT "),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("rT "),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("Route"),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("Rte"),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("Rt"),"location_route"] = 1

df["location_00000"] = 0
df.loc[df["fine_grained_location"].str.contains("000"),"location_00000"] = 1

In [163]:
df.loc[df["location_raw"]=="e lyme","location_raw"] = "east lyme"
df.loc[df["location_raw"]=="e hartford","location_raw"] = "east hartford"
df.loc[df["location_raw"]=="e granby","location_raw"] = "east granby"
df.loc[df["location_raw"]=="no stonington","location_raw"] = "north stonington"
df.loc[df["location_raw"]=="no canaan","location_raw"] = "north canaan"

In [164]:
df["location_raw"].unique()
location_list = np.array(['berlin', 'greenwich', 'brooklyn', 'essex', 'union', 'shelton',
       'naugatuck', 'manchester', 'meriden', 'bridgeport', 'chester',
       'montville', 'oxford', 'beacon falls', 'waterford', 'ledyard',
       'middlefield', 'southbury', 'killingly', 'andover', 'colchester',
       'trumbull', 'bozrah', 'windsor', 'westport', 'litchfield',
       'coventry', 'plainfield', 'thompson', 'stonington', 'stafford',
       'middletown', 'harwinton', 'stratford', 'columbia',
       'new haven', 'mansfield', 'griswold', 'windham', 'hebron',
       'old saybrook', 'west haven', 'torrington', 'preston', 'sterling',
       'north stonington', 'vernon', 'deep river', 'hartford',
       'brookfield', 'darien', 'enfield', 'new britain', 'old lyme',
       'norwalk', 'willington', 'branford', 'wethersfield', 'sherman',
       'marlborough', 'somers', 'hamden', 'waterbury', 'barkhamsted',
       'stamford', 'clinton', 'ashford', 'east haddam', 'danbury',
       'lebanon', 'southington', 'watertown', 'milford', 'tolland',
       'windsor locks', 'madison', 'orange', 'westbrook', 'putnam',
       'north haven', 'ellington', 'burlington', 'franklin',
       'bridgewater', 'groton', 'fairfield', 'new hartford', 'bolton',
       'guilford', 'wallingford', 'newtown', 'east hartford', 'granby',
       'woodbridge', 'farmington', 'new canaan', 'norwich', 'voluntown',
       'sharon', 'durham', 'haddam', 'rocky hill', 'salem', 'bethlehem',
       'seymour', 'winchester', 'thomaston', 'lisbon', 'roxbury',
       'cromwell', 'east granby', 'west hartford', 'woodbury',
       'middlebury', 'cornwall', 'east windsor', 'north canaan',
       'cheshire', 'norfolk', 'east lyme', 'derby', 'newington',
       'woodstock', 'goshen', 'new milford', 'bethany', 'scotland',
       'colebrook', 'prospect', 'glastonbury', 'hampton', 'sprague',
       'canterbury', 'new london', 'bethel', 'morris', 'salisbury',
       'pomfret', 'north branford', 'east haven', 'kent', 'ansonia',
       'killingworth', 'chaplin', 'plainville', 'bloomfield',
       'canaan', 'south windsor', 'washighton', 'hartland', 'eastford',
       'new fairfield', 'warren', 'lyme', 'bristol',
       'plymouth', 'easton'],
      dtype=object)

In [165]:
df["officer_id"].unique()[df["officer_id"].value_counts()>1000]

array([], dtype=int64)

In [166]:
df["officer_id"].unique()[df["officer_id"].value_counts()>50]
officer_list = np.array(['55887736', '770626977', '1000002961', '1000002376', '70583331',
       '1000002714', '1000003026', '1000002219', '298463910',
       '1000002220', '1000003191'],
      dtype=object)

In [167]:
%%time
for list in location_list:
    df["location_"+list] = np.zeros(df.contraband_found.count())
    df.loc[df["location_raw"]==list,"location_"+list]=1

CPU times: user 35.3 s, sys: 5.75 s, total: 41.1 s
Wall time: 10.8 s


In [168]:
%%time
for list in officer_list:
    df["officer_"+str(list)] = np.zeros(df.contraband_found.count())
    df.loc[df["location_raw"]==str(list),"officer_"+str(list)]=1

CPU times: user 3.75 s, sys: 832 ms, total: 4.58 s
Wall time: 1.15 s


In [169]:
df["location_I84"] = 0
df.loc[(df["fine_grained_location"].str.contains("84")),"location_I84"] = 1

df["location_I91"] = 0
df.loc[(df["fine_grained_location"].str.contains("91")) & ~(df["fine_grained_location"].str.contains("T091")),"location_I91"] = 1

df["location_I95"] = 0
df.loc[(df["fine_grained_location"].str.contains("95")) & ~(df["fine_grained_location"].str.contains("T095")),"location_I95"] = 1

df["location_I395"] = 0
df.loc[(df["fine_grained_location"].str.contains("395")) & ~(df["fine_grained_location"].str.contains("T395")),"location_I395"] = 1

In [170]:
df = df.drop(["fine_grained_location","location_raw","officer_id"],axis=1)

In [171]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93994 entries, 0 to 93993
Columns: 252 entries, driver_gender to location_I395
dtypes: float64(179), int64(12), uint8(61)
memory usage: 142.4 MB


In [172]:
df.to_csv('test_clean_06131800.csv',sep=',',index=False)